In [ ]:
from pathlib import Path

import duckdb
import quackosm as qosm
from lonboard import viz
from pooch import retrieve
import shutil
from shapely import box

In [ ]:
duckdb.install_extension("spatial")
duckdb.load_extension("spatial")

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Preparations

- Downloading Tokyo PBF file from BBBike
- Applying geometry filter
- Selecting amenities and primary highways and removing all buildings
- Running QuackOSM with `debug_memory` mode

In [ ]:
TOKYO_DOWNLOAD_URL = "https://download.bbbike.org/osm/bbbike/Tokyo/Tokyo.osm.pbf"
TOKYO_PBF_PATH = retrieve(
    url=TOKYO_DOWNLOAD_URL,
    path=Path(".").resolve(),
    known_hash=None,
)
GEOMETRY_FILTER = box(xmin=139.686, xmax=139.748, ymin=35.618, ymax=35.673)
TAGS_FILTER = {"amenity": True, "building": False, "highway": "primary"}

DEBUG_DIRECTORY = Path("quackosm_debug")
if DEBUG_DIRECTORY.exists():
    shutil.rmtree(DEBUG_DIRECTORY)

result_file = qosm.convert_pbf_to_parquet(
    pbf_path=TOKYO_PBF_PATH,
    debug_memory=True,
    working_directory=DEBUG_DIRECTORY,
    geometry_filter=GEOMETRY_FILTER,
    tags_filter=TAGS_FILTER,
)

### Raw OSM file

#### Nodes

In [ ]:
duckdb.sql(
    f"SELECT * FROM ST_ReadOSM('{TOKYO_PBF_PATH}') WHERE kind = 'node' LIMIT 10"
).show(max_width=160)

#### Ways

In [ ]:
duckdb.sql(
    f"SELECT * FROM ST_ReadOSM('{TOKYO_PBF_PATH}') WHERE kind = 'way' LIMIT 10"
).show(max_width=160)

#### Relations

In [ ]:
duckdb.sql(
    f"SELECT * FROM ST_ReadOSM('{TOKYO_PBF_PATH}') WHERE kind = 'relation' LIMIT 10"
).show(max_width=160)

### Step 1. Reading nodes

In [ ]:
nodes_valid_with_tags_path = next(DEBUG_DIRECTORY.glob("**/nodes_valid_with_tags/**"))
duckdb.sql(f"SELECT * FROM '{nodes_valid_with_tags_path}/*.parquet'")

### Step 2. Filtering nodes - intersection

IDs of nodes filtered by geometry - simple intersection within geometry filter

In [ ]:
nodes_intersecting_ids_path = next(DEBUG_DIRECTORY.glob("**/nodes_intersecting_ids/**"))
duckdb.sql(f"SELECT * FROM '{nodes_intersecting_ids_path}/*.parquet' LIMIT 10")

In [ ]:
viz(
    duckdb.sql(
        f"SELECT tags, ST_Point2D(lon, lat) FROM '{nodes_intersecting_ids_path}/*.parquet' JOIN '{nodes_valid_with_tags_path}/*.parquet' USING (id)"
    )
)

### Step 3. Filtering nodes - tags

IDs of nodes filtered by tags

In [ ]:
nodes_filtered_ids_path = next(DEBUG_DIRECTORY.glob("**/nodes_filtered_ids/**"))
duckdb.sql(f"SELECT * FROM '{nodes_filtered_ids_path}/*.parquet' LIMIT 10")

In [ ]:
viz(
    duckdb.sql(
        f"SELECT tags, ST_Point2D(lon, lat) FROM '{nodes_filtered_ids_path}/*.parquet' JOIN '{nodes_valid_with_tags_path}/*.parquet' USING (id)"
    )
)

### Step 5. Reading ways

- `tags` are used for filtering features
- `raw_tags` are used later to properly detect if way is a polygon

In [ ]:
ways_all_with_tags_path = next(DEBUG_DIRECTORY.glob("**/ways_all_with_tags/**"))
duckdb.sql(f"SELECT * FROM '{ways_all_with_tags_path}/*.parquet' LIMIT 10")

### Step 6. Unnesting ways

Splitting nodes IDs from a list to separate rows with `ref_idx` used for proper ordering

In [ ]:
ways_with_unnested_nodes_refs_path = next(DEBUG_DIRECTORY.glob("**/ways_with_unnested_nodes_refs/**"))
duckdb.sql(f"SELECT * FROM '{ways_with_unnested_nodes_refs_path}/*.parquet' LIMIT 10")

### Step 7. Filtering ways - valid refs

IDs of ways with all refs matched with nodes - removed all ways for which we can't find nodes in the file

In [ ]:
ways_valid_ids_path = next(DEBUG_DIRECTORY.glob("**/ways_valid_ids/**"))
duckdb.sql(f"SELECT * FROM '{ways_valid_ids_path}/*.parquet' LIMIT 10")

### Step 8. Filtering ways - intersection
IDs of ways where at least single node is within given geometry filter - simple join with intersecting nodes

In [ ]:
ways_intersecting_ids_path = next(DEBUG_DIRECTORY.glob("**/ways_intersecting_ids/**"))
duckdb.sql(f"SELECT * FROM '{ways_intersecting_ids_path}/*.parquet' LIMIT 10")

In [ ]:
viz(
    duckdb.sql(
        f"""
        SELECT ways_tags.tags, ST_Point2D(nodes.lon, nodes.lat)
        FROM '{ways_intersecting_ids_path}/*.parquet'
        JOIN '{ways_all_with_tags_path}/*.parquet' ways_tags USING (id)
        JOIN '{ways_with_unnested_nodes_refs_path}/*.parquet' ways USING (id)
        JOIN '{nodes_valid_with_tags_path}/*.parquet' nodes ON ways.ref = nodes.id
        """
    )
)

### Step 9. Filtering ways - tags
IDs of ways filtered by tags

In [ ]:
ways_filtered_ids_path = next(DEBUG_DIRECTORY.glob("**/ways_filtered_ids/**"))
duckdb.sql(f"SELECT * FROM '{ways_filtered_ids_path}/*.parquet' LIMIT 10")

In [ ]:
viz(
    duckdb.sql(
        f"""
        SELECT ways_tags.tags, ST_Point2D(nodes.lon, nodes.lat)
        FROM '{ways_filtered_ids_path}/*.parquet'
        JOIN '{ways_all_with_tags_path}/*.parquet' ways_tags USING (id)
        JOIN '{ways_with_unnested_nodes_refs_path}/*.parquet' ways USING (id)
        JOIN '{nodes_valid_with_tags_path}/*.parquet' nodes ON ways.ref = nodes.id
        """
    )
)

### Step 11. Reading relations

In [ ]:
relations_all_with_tags_path = next(DEBUG_DIRECTORY.glob("**/relations_all_with_tags/**"))
duckdb.sql(f"SELECT * FROM '{relations_all_with_tags_path}/*.parquet' LIMIT 10")

### Step 12. Unnesting relations
Splitting ways IDs from a list to separate rows with ref_role to show linearring role in polygon and `ref_idx` used for proper ordering.

In [ ]:
relations_with_unnested_way_refs_path = next(DEBUG_DIRECTORY.glob("**/relations_with_unnested_way_refs/**"))
duckdb.sql(f"SELECT * FROM '{relations_with_unnested_way_refs_path}/*.parquet' LIMIT 10")

### Step 13. Filtering relations - valid refs

IDs of relations with all refs matched with ways - removed all relations for which we can't find ways in the file

In [ ]:
relations_valid_ids_path = next(DEBUG_DIRECTORY.glob("**/relations_valid_ids/**"))
duckdb.sql(f"SELECT * FROM '{relations_valid_ids_path}/*.parquet' LIMIT 10")

### Step 14. Filtering relations - intersection

IDs of relations where at least single way is within given geometry filter - simple join with intersecting ways

In [ ]:
relations_intersecting_ids_path = next(DEBUG_DIRECTORY.glob("**/relations_intersecting_ids/**"))
duckdb.sql(f"SELECT * FROM '{relations_intersecting_ids_path}/*.parquet' LIMIT 10")

In [ ]:
viz(
    duckdb.sql(
        f"""
        SELECT relations_tags.tags, ST_Point2D(nodes.lon, nodes.lat)
        FROM '{relations_intersecting_ids_path}/*.parquet'
        JOIN '{relations_all_with_tags_path}/*.parquet' relations_tags USING (id)
        JOIN '{relations_with_unnested_way_refs_path}/*.parquet' relations USING (id)
        JOIN '{ways_with_unnested_nodes_refs_path}/*.parquet' ways ON relations.ref = ways.id
        JOIN '{nodes_valid_with_tags_path}/*.parquet' nodes ON ways.ref = nodes.id
        """
    )
)

### Step 15. Filtering relations - tags

IDs of relations filtered by tags

In [ ]:
relations_filtered_ids_path = next(DEBUG_DIRECTORY.glob("**/relations_filtered_ids/**"))
duckdb.sql(f"SELECT * FROM '{relations_filtered_ids_path}/*.parquet' LIMIT 10")

In [ ]:
viz(
    duckdb.sql(
        f"""
        SELECT relations_tags.tags, ST_Point2D(nodes.lon, nodes.lat)
        FROM '{relations_filtered_ids_path}/*.parquet'
        JOIN '{relations_all_with_tags_path}/*.parquet' relations_tags USING (id)
        JOIN '{relations_with_unnested_way_refs_path}/*.parquet' relations USING (id)
        JOIN '{ways_with_unnested_nodes_refs_path}/*.parquet' ways ON relations.ref = ways.id
        JOIN '{nodes_valid_with_tags_path}/*.parquet' nodes ON ways.ref = nodes.id
        """
    )
)

### Step 17. Loading required ways

Ways IDs required by filtered relations

In [ ]:
ways_required_ids_path = next(DEBUG_DIRECTORY.glob("**/ways_required_ids/**"))
duckdb.sql(f"SELECT * FROM '{ways_required_ids_path}/*.parquet' LIMIT 10")

### Step 19. Saving filtered nodes with geometries

Saving prepated geometries in  WKB format with final `feature_id` format and prepared tags

In [ ]:
filtered_nodes_with_geometry_path = next(DEBUG_DIRECTORY.glob("**/filtered_nodes_with_geometry/**"))
duckdb.sql(f"SELECT * FROM '{filtered_nodes_with_geometry_path}/*.parquet' LIMIT 10")

In [ ]:
viz(
    duckdb.sql(
        f"SELECT feature_id, tags, ST_GeomFromWKB(geometry_wkb) geom FROM '{filtered_nodes_with_geometry_path}/*.parquet'"
    )
)

### Step 20. Grouping filtered ways
Because grouping is most resource-intensive operation, data is split into buckets with maximal number of rows dependent on total available memory in the system.

|     Memory | Rows per group |
| ---------: | -------------: |
|     < 8 GB |        100 000 |
|  8 - 16 GB |        500 000 |
| 16 - 24 GB |      1 000 000 |
|    > 24 GB |      5 000 000 |

In [ ]:
filtered_ways_grouped_path = next(DEBUG_DIRECTORY.glob("**/filtered_ways_grouped/group=0/**"))
duckdb.sql(f"SELECT * FROM '{filtered_ways_grouped_path}/*.parquet' ORDER BY id, ref_idx LIMIT 10")

### Step 21. Saving filtered ways with linestrings

Grouping of nodes into a list of points in order is executed per each group assigned previously.

Grouping operation is executed in separate thread, detects `OutOfMemory` errors and automatically retries the operation with smaller number of rows.

In [ ]:
filtered_ways_with_linestrings_path = next(DEBUG_DIRECTORY.glob("**/filtered_ways_with_linestrings/group=0/**"))
duckdb.sql(f"SELECT * FROM '{filtered_ways_with_linestrings_path}/*.parquet' LIMIT 10")

### Step 22. Grouping required ways

The same operation as step 20, but applied on `required` ways.

In [ ]:
required_ways_grouped_path = next(DEBUG_DIRECTORY.glob("**/required_ways_grouped/group=0/**"))
duckdb.sql(f"SELECT * FROM '{required_ways_grouped_path}/*.parquet' ORDER BY id, ref_idx LIMIT 10")

### Step 23. Saving required ways with linestrings

The same operation as step 21, but applied on `required` ways.

In [ ]:
required_ways_with_linestrings_path = next(DEBUG_DIRECTORY.glob("**/required_ways_with_linestrings/group=0/**"))
duckdb.sql(f"SELECT * FROM '{required_ways_with_linestrings_path}/*.parquet' LIMIT 10")

### Step 24. Saving filtered ways with geometries

Saving `filtered` ways as linestrings or as polygons, based on a polygon config.

Linestring has to have at least 3 points, start and end point have to be the same and tags config have to define object as polygon.

In [ ]:
filtered_ways_with_geometry_path = next(DEBUG_DIRECTORY.glob("**/filtered_ways_with_geometry/**"))
duckdb.sql(f"SELECT * FROM '{filtered_ways_with_geometry_path}/*.parquet' LIMIT 10")

In [ ]:
viz(
    duckdb.sql(
        f"SELECT feature_id, tags, ST_GeomFromWKB(geometry_wkb) geom FROM '{filtered_ways_with_geometry_path}/*.parquet'"
    )
)

### Step 25. Saving valid relations parts

Collecting all ways linestrings related to the relation and merging them using `ST_LineMerge` function.

If any linestring after this operation has less than 4 points or isn't closed - relation is discarded.

In [ ]:
valid_relation_parts_path = next(DEBUG_DIRECTORY.glob("**/valid_relation_parts/**"))
duckdb.sql(f"SELECT * FROM '{valid_relation_parts_path}/*.parquet' LIMIT 10")

In [ ]:
viz(
    duckdb.sql(
        f"SELECT id, ref_role, geometry_id, ST_GeomFromWKB(geometry_wkb) geom FROM '{valid_relation_parts_path}/*.parquet'"
    )
)

### Step 26. Saving relations inner parts

Saving `inner` polygons in separate file.

In [ ]:
relation_inner_parts_path = next(DEBUG_DIRECTORY.glob("**/relation_inner_parts/**"))
duckdb.sql(f"SELECT * FROM '{relation_inner_parts_path}/*.parquet' LIMIT 10")

### Step 27. Saving relations outer parts

Saving `outer` polygons in separate file.

In [ ]:
relation_outer_parts_path = next(DEBUG_DIRECTORY.glob("**/relation_outer_parts/**"))
duckdb.sql(f"SELECT * FROM '{relation_outer_parts_path}/*.parquet' LIMIT 10")

### Step 28. Saving relations outer parts with holes

Joining each `outer` polygon with `inner` polygon with `ST_WITHIN` predicate and removing the `inner` polygon from the `outer`.

In [ ]:
relation_outer_parts_with_holes_path = next(DEBUG_DIRECTORY.glob("**/relation_outer_parts_with_holes/**"))
duckdb.sql(f"SELECT * FROM '{relation_outer_parts_with_holes_path}/*.parquet' LIMIT 10")

### Step 29. Saving relations outer parts without holes

Saving all `outer` polygons without holes in a separate file.

In [ ]:
relation_outer_parts_without_holes_path = next(DEBUG_DIRECTORY.glob("**/relation_outer_parts_without_holes/**"))
duckdb.sql(f"SELECT * FROM '{relation_outer_parts_without_holes_path}/*.parquet' LIMIT 10")

### Step 30. Saving filtered relations with geometries

Merging all polygons per relation ID and saving them as multipolygons.

In [ ]:
filtered_relations_with_geometry_path = next(DEBUG_DIRECTORY.glob("**/filtered_relations_with_geometry/**"))
duckdb.sql(f"SELECT * FROM '{filtered_relations_with_geometry_path}/*.parquet' LIMIT 10")

In [ ]:
viz(
    duckdb.sql(
        f"SELECT feature_id, tags, ST_GeomFromWKB(geometry_wkb) geom FROM '{filtered_relations_with_geometry_path}/*.parquet'"
    )
)

### Step 31. Saving all features

Fixing all geometries using `ST_MakeValid` and joining all nodes, ways and relations together in a single dataset.

Final structure of columns based on tags is generated in this step.

In [ ]:
osm_valid_elements_path = next(DEBUG_DIRECTORY.glob("**/osm_valid_elements/**"))
duckdb.sql(f"SELECT * FROM '{osm_valid_elements_path}/*.parquet' LIMIT 10")

### Step 32. Saving final geoparquet file

Removing columns with nulls only, saving final file with proper GeoParquet specificaion data.

In [ ]:
result_file

In [ ]:
duckdb.sql(f"SELECT * FROM '{result_file}' LIMIT 10")

In [ ]:
viz(
    duckdb.sql(
        f"SELECT * EXCLUDE (geometry), ST_GeomFromWKB(geometry) geom FROM '{result_file}'"
    )
)